In [3]:
# Importing text data

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [4]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + 1 + max_length]
            self.input_ids.append(input_chunk)
            self.target_ids.append(target_chunk)

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]
    
def create_dataloader_v1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(
        text=text, 
        tokenizer=tokenizer, 
        max_length=max_length, 
        stride=stride
    )

    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last
    )

    return dataloader
        

In [7]:
import torch
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

output_dim = 256 # You can also keep it 512
vocabulary_size = 50257 # BPE Tokenizer vocabulary size

embedding_layer = torch.nn.Embedding(vocabulary_size, output_dim)

dataloader = create_dataloader_v1(
    raw_text, 
    batch_size=8, 
    max_length=4, 
    stride=4
)

data_iter = iter(dataloader)

inputs, targets = next(data_iter)

print(inputs)

[tensor([  607,  8941,   262,   198, 22674, 39047,   503,  4628]), tensor([   13,   257,  1654,     1,  1022,   338,   465, 24882]), tensor([ 314, 4636,  301, 5574,   13, 1182, 7721,  379]), tensor([ 2391, 28582,   835,  1660,  5845,  1022,   257,   262])]


In [ ]:
token_embeddings = embedding_layer(inputs)

context_length = 4

pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# in GPT's absolutely positioning layer, initiate the embedding weights randomly so that the model can learn positional encodings
pos_embeddings = pos_embedding_layer(torch.arange(context_length))

# input embeddings for the encoder

input_encodings = token_embeddings + pos_embeddings

TypeError: unsupported operand type(s) for +: 'Embedding' and 'Tensor'